In [1]:
print("Hello")

Hello


In [2]:
import sys
print("Python executable:", sys.executable)
print("Python version:", sys.version)


Python executable: /home/akprajwal/nj/.venv/bin/python
Python version: 3.13.7 (main, Aug 15 2025, 12:34:02) [GCC 15.2.1 20250813]


In [3]:
import sys, subprocess
# Install packages into the SAME Python used by the notebook kernel
subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "pip"]) 
subprocess.check_call([sys.executable, "-m", "pip", "install", "torch", "--index-url", "https://download.pytorch.org/whl/cpu"]) 
subprocess.check_call([sys.executable, "-m", "pip", "install", "transformers", "ipywidgets"]) 
print("Installation commands finished. Restart the kernel if new packages were installed.")


  Using cached pip-25.3-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.3-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 25.2
    Uninstalling pip-25.2:
      Successfully uninstalled pip-25.2
Looking in indexes: https://download.pytorch.org/whl/cpu
Installation commands finished. Restart the kernel if new packages were installed.


In [4]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [5]:
from transformers import GPT2LMHeadModel

In [6]:
# (Deprecated) installation here moved to the dedicated installer cell above
# If you want to run an alternate install, use the installer cell or run:
# python -m pip install torch --index-url https://download.pytorch.org/whl/cpu


In [7]:
# (Deprecated) upgraded transformers via installer cell above
# Avoid running multiple pip calls that target different interpreters


In [8]:
try:
    import torch
    import importlib, sys
    # If transformers was previously imported when torch wasn't available it may have created dummy classes.
    if "transformers" in sys.modules:
        importlib.reload(sys.modules["transformers"])
    from transformers import GPT2LMHeadModel
    # Detect dummy placeholder created by transformers when torch was missing
    if getattr(GPT2LMHeadModel, "is_dummy", False):
        print("transformers' GPT2LMHeadModel is a dummy (torch wasn't detected during initial import).")
        print("Recommendation: restart the kernel after running the installer cell.")
        raise RuntimeError("Dummy GPT2LMHeadModel - restart kernel recommended.")
    print("torch version:", torch.__version__)
    model_hf = GPT2LMHeadModel.from_pretrained("gpt2")  # may download weights
    sd_hf = model_hf.state_dict()
    for k, v in sd_hf.items():
        print(k, v.shape)
except Exception as e:
    print("Error while loading model:", e)
    import sys
    print("Kernel python:", sys.executable)
    print("If torch is missing, run the installer cell and then restart the kernel.")


torch version: 2.9.0+cpu
transformer.wte.weight torch.Size([50257, 768])
transformer.wpe.weight torch.Size([1024, 768])
transformer.h.0.ln_1.weight torch.Size([768])
transformer.h.0.ln_1.bias torch.Size([768])
transformer.h.0.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.0.attn.c_attn.bias torch.Size([2304])
transformer.h.0.attn.c_proj.weight torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias torch.Size([768])
transformer.h.0.ln_2.weight torch.Size([768])
transformer.h.0.ln_2.bias torch.Size([768])
transformer.h.0.mlp.c_fc.weight torch.Size([768, 3072])
transformer.h.0.mlp.c_fc.bias torch.Size([3072])
transformer.h.0.mlp.c_proj.weight torch.Size([3072, 768])
transformer.h.0.mlp.c_proj.bias torch.Size([768])
transformer.h.1.ln_1.weight torch.Size([768])
transformer.h.1.ln_1.bias torch.Size([768])
transformer.h.1.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.1.attn.c_attn.bias torch.Size([2304])
transformer.h.1.attn.c_proj.weight torch.Size([768, 768])
transfor

In [9]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-11-13 22:27:48--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M   959KB/s    in 1.1s    

2025-11-13 22:27:51 (959 KB/s) - ‘input.txt’ saved [1115394/1115394]



In [11]:
with open('input.txt', 'r') as f:
    text = f.read()
data = text[:1000]
print(data[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [13]:
!pip install tiktoken

In [ ]:
import tiktoken
enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode(data)
print(tokens[:24])

[5962, 22307, 25, 198, 8421, 356, 5120, 597, 2252, 11, 3285, 502, 2740, 13, 198, 198, 3237, 25, 198, 5248, 461, 11, 2740, 13, 198, 198, 5962, 22307, 25, 198]


In [ ]:
import torch
order = torch.tensor(tokens[:24+1])  # load one additional token, so that last token of training set get to see what its predicting.
x = order[:-1].view(4, 6)    # x is the input to the transformer, y will be the token it would be trying to predict.
y = order[1:].view(4, 6)
print(x)   #instead of long sequence of text, get them to be in btach of 4 batch and 6 tokens in each.
print(y)

'''input(x) --> output(y)
   5962 --> 22307
   22307 --> 25'''

tensor([[ 5962, 22307,    25,   198,  8421,   356],
        [ 5120,   597,  2252,    11,  3285,   502],
        [ 2740,    13,   198,   198,  3237,    25],
        [  198,  5248,   461,    11,  2740,    13]])
tensor([[22307,    25,   198,  8421,   356,  5120],
        [  597,  2252,    11,  3285,   502,  2740],
        [   13,   198,   198,  3237,    25,   198],
        [ 5248,   461,    11,  2740,    13,   198]])
